# TimeSeries Analysis, Causality Ranking
We'll use some ideas from EDA_02 and EDA_03 notebooks, which simulated a slider effect.

Our goal is to compare the average R² between the RECENT CHANGE in predictor variable (as a %) with the CHANGE in the target variable (as a %), given various deltas.
We'll mark any correlation with p-value of less than 5% as being 0.

This time, we'll use all the new data we've gathered.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
from statistics import mean


pd.set_option('display.max_columns', None)

In [2]:
df_master = pd.read_csv("../Data_Sets/processed/completeData_1995-2022.csv")

# Renaming Economic freedom columns
df_master = df_master.rename(columns={
    'Property Rights':'EF_Property Rights',
    'Government Integrity':'EF_Government Integrity',
    'Judicial Effectiveness':'EF_Judicial Effectiveness',
    'Government Spending':'EF_Government Spending',
    'Tax Burden':'EF_Tax Burden',
    'Fiscal Health':'EF_Fiscal Health',
    'Business Freedom':'EF_Business Freedom',
    'Monetary Freedom':'EF_Monetary Freedom',
    'Labor Freedom':'EF_Labor Freedom',
    'Financial Freedom':'EF_Financial Freedom',
    'Investment Freedom':'EF_Investment Freedom',
    'Trade Freedom':'EF_Trade Freedom'
})

# Calculating the 'EF_Overall Score'
EF_features = ['EF_Property Rights', 'EF_Government Integrity', 'EF_Judicial Effectiveness',
               'EF_Government Spending', 'EF_Tax Burden', 'EF_Fiscal Health',
               'EF_Business Freedom', 'EF_Monetary Freedom', 'EF_Labor Freedom',
               'EF_Financial Freedom', 'EF_Investment Freedom', 'EF_Trade Freedom']

df_master['EF_Overall Score'] = df_master[EF_features].sum(axis=1) / 12

In [3]:
df_master.columns

Index(['Country Name', 'Index Year', 'EF_Property Rights',
       'EF_Government Integrity', 'EF_Judicial Effectiveness',
       'EF_Government Spending', 'EF_Tax Burden', 'EF_Fiscal Health',
       'EF_Business Freedom', 'EF_Monetary Freedom', 'EF_Labor Freedom',
       'EF_Financial Freedom', 'EF_Investment Freedom', 'EF_Trade Freedom',
       'GDP per capita (current USD)', 'Total population', 'Gini',
       'Inflation CPI', 'Real interest rate', 'Labor force size',
       'Trade (% of GDP)', 'Trade in services (% of GDP)',
       'Under-5 mortality rate (per 1k live births)', 'Country Quintile',
       'isLandLocked', 'n_accessToSea', 'Rail Density',
       'Pctg of Rail Electrified', 'Area Size (km2)', 'Expanded EconZone Area',
       'Amount of Ports', 'Distance from Equator', 'Average Temperature (C)',
       'Death rates from disasters', 'H index (Academic Papers)',
       'Arable Land pct', 'Qualified Labor Force pct',
       'Human Development Index', 'Natural Resources', 'Mi

In [4]:
years_delta = np.arange(1,26,1)

### TimeSeriesDeltaChange()
To assist us in this journey, let's create the function 'TimeSeriesDeltaChange'. A summary of it:

**Purpose:** This function computes how recent changes in a predictor variable (like 'EF_Overall Score') relate to changes in a target variable (like 'GDP per capita') over various time ranges.

**Working:**
  - Cleans data and computes an Exponential Moving Average (EMA) for the predictor to calculate 'Recent Change in Predictor'.
  - For each time period (delta), it:
	- Shifts the data by that period.
	- Calculates the percentage change for the target.
	- Fits a polynomial regression model using the difference from EMA as the predictor.

**Output:** The function outputs an augmented DataFrame (df_withDeltas) that contains computed changes for each time period.

#### Function

In [5]:
def TimeSeriesDeltaChange(
        df,
        predictors,
        target = 'GDP per capita (current USD)',
        quantiledTarget = 'Country Quintile',
        timeField = 'Index Year',
        mergeFields = ['Country Name', 'Index Year'],
        timePeriods_delta = [1,4],
        EMAdepth = 3
    ):
    '''
    In a nutshell, this function analyzes time-series data using a time-slider-type analysis.
    
    predictor = the variable(s) name(s) you want to analyze.
    target = the dependent variable name we want to predict for.
    quantiledTarget = categorical data to split the data (useful for later visualization)
    timeField = the name of the time column you're using
    mergeFields = a list to define the Primary/Composite Key of the DataSet.
    timePeriods_delta = list of delta ranges to test the data against
    '''
    
    # Initializing Base DataFrame
    cleaned_data = df[[
                        mergeFields[0],
                        mergeFields[1],
                        *predictors,
                        target,
                        quantiledTarget
                    ]].dropna()
    
    # Inverting negative features, so it's easier to interpret them in aggregation with other features.
    cleaned_data['Gini'] = cleaned_data['Gini'] * - 1
    cleaned_data['Local Mean Gini'] = cleaned_data['Local Mean Gini'] * - 1
    cleaned_data['Inflation CPI'] = cleaned_data['Inflation CPI'] * - 1
    cleaned_data['Death rates from disasters'] = cleaned_data['Death rates from disasters'] * - 1
    cleaned_data['Real interest rate'] = cleaned_data['Real interest rate'] * - 1
    cleaned_data['Under-5 mortality rate (per 1k live births)'] = cleaned_data['Under-5 mortality rate (per 1k live births)'] * - 1

    for predictor in predictors:    
        # Precompute the EMA and EMA diff for the predictor
        cleaned_data[f'{predictor}_EMA_{EMAdepth}'] \
            = cleaned_data.groupby('Country Name')[predictor]\
                .transform(lambda x: x.ewm(span=EMAdepth, adjust=False).mean())   
        
        # Calculating the percentage change from predictor to its EMA
        cleaned_data[f'{predictor}_diff_from_EMA'] \
            = ((cleaned_data[predictor] - cleaned_data[f'{predictor}_EMA_{EMAdepth}'])
                / cleaned_data[f'{predictor}_EMA_{EMAdepth}']) * 100
        
        # Avoid Zero values on diff (Certain regressions break with 0 values here)
        # Further on that, that is because certain fields, like Judicial Effectiveness, have late beginnings.
        cleaned_data[f'{predictor}_diff_from_EMA']\
            = cleaned_data[f'{predictor}_diff_from_EMA'].transform(lambda x: np.random.normal(0, 0.000001) if x == 0 else x)
        
        # We don't need '{predictor}_EMA_{EMAdepth}' anymore, so let's drop it
        cleaned_data = cleaned_data.drop(columns = [f'{predictor}_EMA_{EMAdepth}'])
    
    
    df_withDeltas = cleaned_data.copy() #just initializing, for later appending
    
    # Loop over each year delta, compute & store the merged data, and fit a Regression Model
    for delta in timePeriods_delta:
        
        # ----------------- Time Slider Effect Calculation -----------------
        # Create a shifted dataframe to compute the changes
        shifted_data = cleaned_data[[*mergeFields, target]].copy()
        shifted_data[timeField] -= delta
        
        merged_data = pd.merge(
            cleaned_data,
            shifted_data[[*mergeFields, target]],  # Only include the shifted target variable
            on=mergeFields,
            suffixes=('', f'_plus_{delta}'),
            indicator=False,
            how='inner'
        )
        
        # If there's no data for this range, go to the next iteration
        if merged_data.shape[0] == 0:
            print(f"No overlapping data for any of the predictors with a time delta of {delta}. Skipping this delta.")
            continue
        
        # Compute the percentage change for target
        merged_data[f'{target}_change_{delta}']\
            = ((merged_data[f'{target}_plus_{delta}']\
               - merged_data[target])\
               / merged_data[target]) * 100
        
        
        # Merge results to df_withDeltas based on mergeFields
        df_withDeltas = pd.merge(
            df_withDeltas,
            merged_data[[*mergeFields, f'{target}_change_{delta}']],
            on=mergeFields,
            how='left'
        )
    
    return df_withDeltas

#### Generating the DataFrame

In [6]:
# Selecting numeric columns
predictorsList = list(df_master.select_dtypes(include='number').columns)

# Removing Time and Target Variables
predictorsList.remove('Index Year')
predictorsList.remove('GDP per capita (current USD)')

# Removing variables that don't change through time (in our database)
# They tend to be variables with low variability
for col in predictorsList[13:]: # Skipping EF
    if df_master[col].nunique() < 200:
        predictorsList.remove(col)


df_withDeltas = TimeSeriesDeltaChange(df_master, predictors=predictorsList, timePeriods_delta=years_delta)

#### Generating Stats from DF
We'll iterate over each predictor, collect stats for all delta, and store the average value.

For each iteration, we'll use the highest score from different Polynomial Degrees, to capture possible non-linear relationships

In [7]:
df_withDeltas.fillna(0, inplace=True)

In [8]:
def meanR2ForLaggedValues(df_withDeltas = df_withDeltas, predictorsList = predictorsList, years_delta = years_delta, quintiled = None):

    '''
        For each feature, calculates the best fit in a polynomial (from degree 1 trough 6), and returns the mean R² score for all Delta Years.

        df_withDeltas: DataFrame with lagged values data.
        predictorsList: List of predictors we'll analyze
        years_delta: List of lagged values we'll analyze
        quintiled: selects quantile data from df_withDeltas
    '''

    # If True, selecting only the specified quintiled portion
    if quintiled:
        df_withDeltas = df_withDeltas[df_withDeltas['Country Quintile'] == quintiled]

    POLYORDER = np.arange(1,7)
    dict_r2_scores = {}

    for predictor in predictorsList:
        predictor_r2_scores = []

        for degree in POLYORDER:
            max_r2_scores = 0

            for delta in years_delta:
                # -------------------- Model Building & Fitting --------------------
                X = df_withDeltas[f'{predictor}_diff_from_EMA'].values.reshape(-1, 1)   
                y = df_withDeltas[f'GDP per capita (current USD)_change_{delta}']

                # Polynomial Regression
                poly = PolynomialFeatures(degree=degree)            
                X_poly = poly.fit_transform(X)

                # Using statsmodels to fit the model
                model = sm.OLS(y, X_poly).fit()
                R2_score = model.rsquared
                #display(R2_score)

                # Given p-value is less than 5%, store R²
                if (model.pvalues < 0.05).all() and R2_score > max_r2_scores:
                    max_r2_scores = R2_score



            predictor_r2_scores.append(max_r2_scores)
        
        if predictor_r2_scores:
            dict_r2_scores[predictor] = mean(predictor_r2_scores)
        else:
            dict_r2_scores[predictor] = None


    # Result DataFrame
    result_df = pd.DataFrame(list(dict_r2_scores.items()), columns=['Predictor', 'R2 Scores']).sort_values('R2 Scores', ascending=False).reset_index(drop=True)

    # Defining categories for each feature, so we can groupBy later
    dict_featureGroupings = {
        'geoPolitical':
            ['Natural Resources','Arable Land pct','Migration Volume',
            'Population Density', 'Total population'],

        'Economic Freedom':
            ['EF_Property Rights', 'EF_Government Integrity',
        'EF_Judicial Effectiveness', 'EF_Government Spending', 'EF_Tax Burden',
        'EF_Fiscal Health', 'EF_Business Freedom', 'EF_Monetary Freedom',
        'EF_Labor Freedom', 'EF_Financial Freedom', 'EF_Investment Freedom',
        'EF_Trade Freedom', 'EF_Overall Score'],

        'Other Economic Factors':
            ['Gini', 'Inflation CPI', 'Real interest rate', 'Labor force size',
        f'Trade (% of GDP)',f'Trade in services (% of GDP)', 'Under-5 mortality rate (per 1k live births)',
        'Death rates from disasters', 'Human Development Index'],

        'Education & Research':
            ['Qualified Labor Force pct','Harmonized Test Scores'],        

        'Neighbour Quality':
            ['Local Mean GDP per Capita','Local Mean GDP','Local Mean Gini']
    }

    # Inverting the dict to create a mapping from feature to group
    feature_to_group = {feature: group for group, features in dict_featureGroupings.items() for feature in features}

    # Adding the 'Predictor Group' column to the result DataFrame
    result_df['Predictor Group'] = result_df['Predictor'].apply(lambda x: feature_to_group.get(x, 'Unknown')).astype('category')
    result_df['Predictor'] = result_df['Predictor'].astype('category')

    return result_df

## Results

R² Scores are low for every predictor group, which is expected. We're using a rudimentary method prediction-wise. Yet this gives us an important insight on a Causality hypothesis, since all of our data is using lagged values.

As we can see, excluding features we can't/shouldn't directly influence politically (like Mean Neighbour GDP), Economic Freedom is still the most important factor for causing GDP per Capita. (Though Other Economic Factors are not far behind).

Surprisingly, Education & Research features have really low R² Scores.

In [9]:
df_allQuintiles = meanR2ForLaggedValues()

display('Causation Score by Predictor Group, for the whole Data')
df_allQuintiles.groupby('Predictor Group', observed=False)['R2 Scores'].mean().reset_index(drop=False).sort_values('R2 Scores', ascending=False)


'Causation Score by Predictor Group, for the whole Data'

,Predictor Group,R2 Scores
2,Neighbour Quality,0.037543
0,Economic Freedom,0.017451
3,Other Economic Factors,0.012229
4,geoPolitical,0.011138
1,Education & Research,0.001263


But wait. Simpson's Paradox could be at play here, so let's run the same data again, groupedBy quintiles.

We'll average all results, so we can easily compare it with the previous table.

In [10]:
quintileList = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']

df_quintiled = pd.DataFrame(columns=['Predictor Group', 'R2 Scores'])

for q in quintileList:
    #display(f'Causation Score by predictor Group, for Quintile: {q}')

    # If you want to see each quintile data separately, uncomment the code below:
    #display(meanR2ForLaggedValues(quintiled=q).groupby('Predictor Group', observed=False)['R2 Scores'].mean().reset_index(drop=False).sort_values('R2 Scores', ascending=False))

    # merge quintile data into a single dataFrame
    newQuant = meanR2ForLaggedValues(quintiled=q)
    df_quintiled = df_quintiled.merge(newQuant, how='outer', on='Predictor Group', suffixes=('', f'_{q}'))
    
display('Mean Causation Score by Predictor Group, groupedBy each Quintile')
# Calculating the Mean R2 Scores from Quintiled Data
df_quintiled['Mean R2 Scores'] = (df_quintiled['R2 Scores_Q1'] +
                                  df_quintiled['R2 Scores_Q2'] +
                                  df_quintiled['R2 Scores_Q3'] +
                                  df_quintiled['R2 Scores_Q4'] +
                                  df_quintiled['R2 Scores_Q5']) / 5

df_quintiled[['Predictor Group','Mean R2 Scores']].groupby('Predictor Group', observed=False)['Mean R2 Scores'].mean().reset_index(drop=False).sort_values('Mean R2 Scores', ascending=False)


'Mean Causation Score by Predictor Group, groupedBy each Quintile'

,Predictor Group,Mean R2 Scores
2,Neighbour Quality,0.048644
4,geoPolitical,0.025114
0,Economic Freedom,0.016855
3,Other Economic Factors,0.015439
1,Education & Research,0.002103


In [11]:
df_quintiled.groupby('Predictor Group', observed=False)[['R2 Scores_Q1','R2 Scores_Q2','R2 Scores_Q3','R2 Scores_Q4','R2 Scores_Q5','Mean R2 Scores']]\
        .mean().reset_index(drop=False).sort_values('Mean R2 Scores', ascending=False)

,Predictor Group,R2 Scores_Q1,R2 Scores_Q2,R2 Scores_Q3,R2 Scores_Q4,R2 Scores_Q5,Mean R2 Scores
2,Neighbour Quality,0.042012,0.054810,0.043929,0.050250,0.052219,0.048644
4,geoPolitical,0.034035,0.035805,0.011837,0.018265,0.025630,0.025114
0,Economic Freedom,0.011576,0.021257,0.024189,0.016628,0.010628,0.016855
3,Other Economic Factors,0.010213,0.019635,0.019537,0.015884,0.011925,0.015439
1,Education & Research,0.004537,0.000000,0.001203,0.000000,0.004776,0.002103


In [12]:
melted_df_quintiled = pd.melt(df_quintiled[['Predictor Group','R2 Scores_Q1','R2 Scores_Q2','R2 Scores_Q3','R2 Scores_Q4','R2 Scores_Q5']], id_vars=['Predictor Group'], var_name='Description', value_name='Value')

In [13]:
melted_df_quintiled['Value'] = melted_df_quintiled['Value'].round(5)
melted_df_quintiled = melted_df_quintiled.drop_duplicates(subset=['Value'])

In [14]:
# Exporting data to flourish
df_allQuintiles.groupby('Predictor Group', observed=False)['R2 Scores'].mean().reset_index(drop=False).sort_values('R2 Scores', ascending=False).to_csv("../Data_Sets/toFlourish/causation_allQuintiles.csv",index=False)

melted_df_quintiled.to_csv("../Data_Sets/toFlourish/causation_quintiled.csv",index=False)

The results shift a bit, but the overall conclusion remains.

Economic Freedom is the most important, closely followed by Other Economic Factors, and Education & Research is considerably behind every Predictor Group.